<a href="https://colab.research.google.com/github/Sarthak0205/Corporate-Saathi/blob/main/isl_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from google.colab import drive
drive.mount('/content/drive')

import os

# Path to your dataset
base_dir = "/content/drive/MyDrive/ISL_dataset"
# List first few files/folders
print(os.listdir(base_dir))


import os
import shutil

# Path where your videos are currently stored
src_dir = "/content/drive/MyDrive/ISL_dataset"   # change if needed
# Path where reorganized dataset will be created
dst_dir = "/content/drive/MyDrive/dataset_sorted"

os.makedirs(dst_dir, exist_ok=True)

# List all .MOV files in the source folder
videos = [f for f in os.listdir(src_dir) if f.lower().endswith(".mov")]

print(f"Found {len(videos)} videos.")

# Reorganize into gesture_1, gesture_2, ...
for idx, vid in enumerate(sorted(videos), start=1):
    gesture_folder = os.path.join(dst_dir, f"gesture_{idx}")
    os.makedirs(gesture_folder, exist_ok=True)

    src_path = os.path.join(src_dir, vid)
    dst_path = os.path.join(gesture_folder, vid)

    shutil.copy(src_path, dst_path)  # use copy so original files stay safe
    print(f"✅ Moved {vid} -> {gesture_folder}")

print("\n🎉 Dataset reorganized successfully!")
print(f"Output path: {dst_dir}")


import cv2
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout

# -------------------------------
# SETTINGS
# -------------------------------
IMG_SIZE = 64        # Resize frames
MAX_FRAMES = 30      # Frames per video
DATASET_PATH = "/content/drive/MyDrive/dataset_sorted"  # 👈 reorganized dataset

# -------------------------------
# VIDEO TO ARRAY FUNCTION
# -------------------------------
def video_to_array(path, max_frames=MAX_FRAMES):
    cap = cv2.VideoCapture(path)
    frames = []
    count = 0
    while cap.isOpened() and count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        frame = frame / 255.0   # normalize
        frames.append(frame)
        count += 1
    cap.release()
    # Pad with zeros if shorter
    while len(frames) < max_frames:
        frames.append(np.zeros((IMG_SIZE, IMG_SIZE, 3)))
    return np.array(frames)

# -------------------------------
# LOAD DATASET
# -------------------------------
X, y, labels = [], [], []
classes = sorted(os.listdir(DATASET_PATH))

for idx, gesture in enumerate(classes):
    gesture_path = os.path.join(DATASET_PATH, gesture)
    if not os.path.isdir(gesture_path):
        continue
    for vid in os.listdir(gesture_path):
        vid_path = os.path.join(gesture_path, vid)
        frames = video_to_array(vid_path)
        X.append(frames)
        y.append(idx)
    labels.append(gesture)

X = np.array(X)
y = to_categorical(y, num_classes=len(classes))

print("✅ Dataset ready")
print("X shape:", X.shape)   # (videos, frames, 64, 64, 3)
print("y shape:", y.shape)
print("Labels:", labels)

# -------------------------------
# BUILD MODEL (CNN + LSTM)
# -------------------------------
model = Sequential([
    TimeDistributed(Conv2D(32, (3,3), activation='relu'), input_shape=(MAX_FRAMES, IMG_SIZE, IMG_SIZE, 3)),
    TimeDistributed(MaxPooling2D((2,2))),
    TimeDistributed(Flatten()),
    LSTM(64),
    Dropout(0.5),
    Dense(len(classes), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# -------------------------------
# TRAIN MODEL
# -------------------------------
history = model.fit(X, y, epochs=10, batch_size=4, validation_split=0.2)

# -------------------------------
# SAVE MODEL
# -------------------------------
# -------------------------------
# SAVE MODEL (both formats)
# -------------------------------
save_path = "/content/drive/MyDrive"

# Legacy HDF5 format (for compatibility)
model.save(f"{save_path}/isl_model.h5")
print("✅ Model saved as isl_model.h5")

# New Keras format (recommended going forward)
model.save(f"{save_path}/isl_model.keras")
print("✅ Model also saved as isl_model.keras")




def predict_gesture(video_path, model, labels):
    frames = video_to_array(video_path)
    frames = np.expand_dims(frames, axis=0)  # add batch dimension
    prediction = model.predict(frames)
    class_id = np.argmax(prediction)
    return labels[class_id], prediction[0][class_id]

# -------------------------------
# LOAD & TEST
# -------------------------------
# Load model back (for safety)
model = load_model("isl_model.h5")

# 👇 change this path to any test video from your dataset or new video
test_video = "/content/drive/MyDrive/ISL_dataset/1 hello you how .MOV"

gesture, confidence = predict_gesture(test_video, model, labels)
print(f"🖐 Predicted Gesture: {gesture} (Confidence: {confidence:.2f})")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['1 hello you how .MOV', '3 family big .MOV', '7 marraige today .MOV', '5 boy short .MOV', '6 girl hungry .MOV', '9 uncle friuts get .MOV', '10 granfather sick .MOV', '14 baby cute .MOV', '15 both just seperate .MOV', '16 they apply divorce .MOV', '17 we place see go which .MOV', '18 i home reach very late .MOV', '19 this work my not .MOV', '20 this day i inform you .MOV', '21 we night go out .MOV', '22 we this week holiday .MOV', '23 christmas month celebration .MOV', '24 this year  feb 29 date leap not .MOV', '25 raju first come tanu after.MOV', '28 you work finish .MOV', '30 hot very here .MOV', '26 first him eat after i eat .MOV', '31 food taste good .MOV', '29 furnac machine near teamrature hot .MOV', '32 it cold coffee .MOV', '35 milk health good .MOV', '34 pizza food healthy not .MOV', '36 children milk drink like .MOV', '38 curd sweet .MOV', '39 egg w

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed_21             │ (None, 30, 62, 62, 32) │           896 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_22             │ (None, 30, 31, 31, 32) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_23             │ (None, 30, 30752)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 64)             │     7,889,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 76)             │         4,940 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,894,988 (30.12 MB)

 Trainable params: 7,894,988 (30.12 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 66s 4s/step - accuracy: 0.0044 - loss: 4.7387 - val_accuracy: 0.0000e+00 - val_loss: 4.5279
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 77s 4s/step - accuracy: 0.0000e+00 - loss: 4.6262 - val_accuracy: 0.0000e+00 - val_loss: 4.7049
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 83s 4s/step - accuracy: 0.0104 - loss: 4.2946 - val_accuracy: 0.0000e+00 - val_loss: 4.8130
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 59s 4s/step - accuracy: 0.0000e+00 - loss: 4.4779 - val_accuracy: 0.0000e+00 - val_loss: 4.9085
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 56s 4s/step - accuracy: 0.0044 - loss: 4.2797 - val_accuracy: 0.0000e+00 - val_loss: 4.9709
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 57s 4s/step - accuracy: 0.0000e+00 - loss: 4.2993 - val_accuracy: 0.0000e+00 - val_loss: 4.8756
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 81s 4s/step - accuracy: 0.0057 - loss: 4.2303 - val_accuracy: 0.0000e+00 - val_loss: 4.8329
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 90s 4s/step - accuracy: 0.0000e+00 - los

✅ Model saved as isl_model.h5
✅ Model also saved as isl_model.keras


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'isl_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [18]:
!ls -lh
model.save("/content/drive/MyDrive/isl_model.h5")

from tensorflow.keras.models import load_model

# Load the HDF5 model
model = load_model("/content/drive/MyDrive/isl_model.h5")

# OR load the new Keras format
# model = load_model("/content/drive/MyDrive/isl_model.keras")

print("✅ Model loaded successfully!")



total 8.0K
drwx------ 5 root root 4.0K Aug 28 13:42 drive
drwxr-xr-x 1 root root 4.0K Aug 26 19:08 sample_data


✅ Model loaded successfully!
